#### Рекомендательные системы



ДЗ - построение ассоциативных правил для датасета :
Можно использовать например apriori, association_rules, fpgrowth из mlxtend.frequent_patterns (работают чуть быстрее).
Написать выводы. 


https://www.kaggle.com/datasets/gorkhachatryan01/purchase-behaviour?select=dataset.csv

In [1]:
import math, random
import numpy as np
from collections import defaultdict, Counter
from numpy import dot
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.sparse import csr_matrix
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth


In [2]:

file_path_test ="C:/Users/qiarr/Desktop/task/lesson 22/task/dataset.csv"
data = pd.read_csv(file_path_test, encoding='cp1251', )

In [3]:
data

,pork,sandwich bags,lunch meat,all- purpose,flour,soda,butter,vegetables,beef,aluminum foil,all- purpose.1,dinner rolls,shampoo,all- purpose.2
0,shampoo,hand soap,waffles,vegetables,cheeses,mixes,milk,sandwich bags,laundry detergent,dishwashing liquid/detergent,waffles,individual meals,hand soap,vegetables
1,pork,soap,ice cream,toilet paper,dinner rolls,hand soap,spaghetti sauce,milk,ketchup,sandwich loaves,poultry,toilet paper,ice cream,ketchup
2,juice,lunch meat,soda,toilet paper,all- purpose,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,pasta,tortillas,mixes,hand soap,toilet paper,vegetables,vegetables,paper towels,vegetables,flour,vegetables,pork,poultry,eggs
4,toilet paper,eggs,toilet paper,vegetables,bagels,dishwashing liquid/detergent,cereals,paper towels,laundry detergent,butter,cereals,bagels,paper towels,shampoo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1493,beef,sandwich bags,hand soap,paper towels,paper towels,all- purpose,beef,fruits,coffee/tea,beef,shampoo,mixes,cheeses,juice
1494,dinner rolls,lunch meat,spaghetti sauce,pasta,vegetables,cereals,dinner rolls,soap,milk,eggs,poultry,cheeses,cereals,mixes
1495,lunch meat,eggs,poultry,vegetables,tortillas,beef,beef,individual meals,dishwashing liquid/detergent,shampoo,dishwashing liquid/detergent,dishwashing liquid/detergent,hand soap,sugar
1496,ketchup,milk,poultry,cheeses,soap,toilet paper,yogurt,beef,waffles,sugar,spaghetti sauce,all- purpose,sandwich bags,toilet paper


In [4]:
unique_product = data.stack().unique()

выделяем список всех продуктов

In [5]:
unique_product

array([' shampoo', ' hand soap', ' waffles', ' vegetables', ' cheeses',
       ' mixes', ' milk', ' sandwich bags', ' laundry detergent',
       ' dishwashing liquid/detergent', ' individual meals', ' pork',
       ' soap', ' ice cream', ' toilet paper', ' dinner rolls',
       ' spaghetti sauce', ' ketchup', ' sandwich loaves', ' poultry',
       ' juice', ' lunch meat', ' soda', ' all- purpose', ' ', ' pasta',
       ' tortillas', ' paper towels', ' flour', ' eggs', ' bagels',
       ' cereals', ' butter', ' yogurt', ' coffee/tea', ' fruits',
       ' sugar', ' aluminum foil', ' beef'], dtype=object)

In [6]:
unique_product.shape

(39,)

In [7]:
pd.set_option("display.max_columns", 50) 

In [8]:
data_value = data.values

print(data_value)

[[' shampoo' ' hand soap' ' waffles' ... ' individual meals' ' hand soap'
  ' vegetables']
 [' pork' ' soap' ' ice cream' ... ' toilet paper' ' ice cream'
  ' ketchup']
 [' juice' ' lunch meat' ' soda' ... nan nan nan]
 ...
 [' lunch meat' ' eggs' ' poultry' ... ' dishwashing liquid/detergent'
  ' hand soap' ' sugar']
 [' ketchup' ' milk' ' poultry' ... ' all- purpose' ' sandwich bags'
  ' toilet paper']
 [' laundry detergent' ' vegetables' ' shampoo' ... nan nan nan]]


In [9]:
data_value

array([[' shampoo', ' hand soap', ' waffles', ..., ' individual meals',
        ' hand soap', ' vegetables'],
       [' pork', ' soap', ' ice cream', ..., ' toilet paper',
        ' ice cream', ' ketchup'],
       [' juice', ' lunch meat', ' soda', ..., nan, nan, nan],
       ...,
       [' lunch meat', ' eggs', ' poultry', ...,
        ' dishwashing liquid/detergent', ' hand soap', ' sugar'],
       [' ketchup', ' milk', ' poultry', ..., ' all- purpose',
        ' sandwich bags', ' toilet paper'],
       [' laundry detergent', ' vegetables', ' shampoo', ..., nan, nan,
        nan]], dtype=object)

создание разряженной матрицы V3 с помощью pandas и при этом сразу являющейся датафреймом

In [10]:

data_value_list = data_value.tolist()

# Создаем DataFrame из списков покупок
df = pd.DataFrame({'user_id': range(1, len(data_value_list ) + 1), 'items': data_value_list })
df['items'] = df['items'].apply(lambda x: [item if pd.notna(item) else 'nothing' for item in x])
# Инициализируем MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Преобразуем списки в разреженную матрицу
sparse_matrix = csr_matrix(mlb.fit_transform(df['items']))

# Преобразуем разреженную матрицу в DataFrame
sparse_df = pd.DataFrame.sparse.from_spmatrix(sparse_matrix, columns=mlb.classes_, index=df['user_id'])

print(sparse_df)


             all- purpose   aluminum foil   bagels   beef   butter   cereals  \
user_id                                                                        
1        0              0               0        0      0        0         0   
2        0              0               0        0      0        0         0   
3        1              1               0        0      0        0         0   
4        0              0               0        0      0        0         0   
5        0              0               0        1      0        1         1   
...     ..            ...             ...      ...    ...      ...       ...   
1494     0              1               0        0      1        0         0   
1495     0              0               0        0      0        0         1   
1496     0              0               0        0      1        0         0   
1497     0              1               0        0      1        0         0   
1498     1              0               

In [11]:
sparse_df

,,all- purpose,aluminum foil,bagels,beef,butter,cereals,cheeses,coffee/tea,dinner rolls,dishwashing liquid/detergent,eggs,flour,fruits,hand soap,ice cream,individual meals,juice,ketchup,laundry detergent,lunch meat,milk,mixes,paper towels,pasta,pork,poultry,sandwich bags,sandwich loaves,shampoo,soap,soda,spaghetti sauce,sugar,toilet paper,tortillas,vegetables,waffles,yogurt,nothing
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,1,1,0,1,0,1,0,1,0,1,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,0,0,0
5,0,0,0,1,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1494,0,1,0,0,1,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
1495,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,1,1,1,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0
1496,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,1,0,0,0


In [12]:
from collections import Counter

# Создайте список для всех интересов
all_interests = []

# Перебирайте строки в вашем датафрейме
for row in range(len(data)):
    # Используйте .iloc[row] для доступа к строке по индексу
    interests = data.iloc[row].tolist()
    # Добавьте интересы в общий список
    all_interests.extend(interests)

# Используйте Counter для подсчета частоты каждого интереса
popular_interests = Counter(all_interests).most_common()

print(popular_interests)

[(nan, 2229), (' vegetables', 1361), (' poultry', 506), (' ', 496), (' mixes', 488), (' bagels', 486), (' waffles', 482), (' ice cream', 482), (' soda', 482), (' cereals', 480), (' toilet paper', 478), (' lunch meat', 471), (' milk', 470), (' individual meals', 467), (' pasta', 466), (' butter', 465), (' sugar', 465), (' soap', 461), (' aluminum foil', 460), (' cheeses', 456), (' laundry detergent', 456), (' coffee/tea', 456), (' beef', 456), (' eggs', 453), (' fruits', 453), (' flour', 450), (' dishwashing liquid/detergent', 449), (' spaghetti sauce', 447), (' yogurt', 447), (' all- purpose', 443), (' paper towels', 441), (' sandwich bags', 439), (' shampoo', 438), (' sandwich loaves', 437), (' dinner rolls', 435), (' juice', 433), (' pork', 431), (' ketchup', 428), (' tortillas', 426), (' hand soap', 403)]


как видно vegetables есть почти в каждой покупке в магазине

убираем из разряженной матрицы столбец с NaN и  столбец с пустым пространством

In [13]:
sparse_df = sparse_df.drop(columns=['nothing'])
sparse_df = sparse_df.drop(columns=[' '])
 


In [14]:


# Применение алгоритма apriori
frequent_itemsets_apriori = apriori(sparse_df, min_support=0.01, use_colnames=True)

# Получение ассоциативных правил с помощью apriori
rules_apriori = association_rules(frequent_itemsets_apriori, metric="confidence", min_threshold=0.7)

# Применение алгоритма fpgrowth
frequent_itemsets_fpgrowth = fpgrowth(sparse_df, min_support=0.01, use_colnames=True)

# Получение ассоциативных правил с помощью fpgrowth
rules_fpgrowth = association_rules(frequent_itemsets_fpgrowth, metric="confidence", min_threshold=0.7)

# Вывод результатов
print("Ассоциативные правила с помощью apriori:")
print(rules_apriori.head)

print("\nАссоциативные правила с помощью fpgrowth:")
print(rules_fpgrowth.head)

c:\Users\qiarr\AppData\Local\Programs\Python\Python310\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


c:\Users\qiarr\AppData\Local\Programs\Python\Python310\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Ассоциативные правила с помощью apriori:
<bound method NDFrame.head of                                antecedents    consequents  antecedent support  \
0           ( laundry detergent,  cereals)  ( vegetables)            0.074099   
1           ( laundry detergent,  cheeses)  ( vegetables)            0.070093   
2                      ( yogurt,  cheeses)  ( vegetables)            0.070761   
3                ( eggs,  sandwich loaves)  ( vegetables)            0.060748   
4             ( laundry detergent,  flour)  ( vegetables)            0.071429   
..                                     ...            ...                 ...   
767     ( spaghetti sauce,  yogurt,  soap)  ( vegetables)            0.016021   
768     ( spaghetti sauce,  yogurt,  soda)  ( vegetables)            0.019359   
769           ( yogurt,  tortillas,  soda)  ( vegetables)            0.018692   
770  ( spaghetti sauce,  yogurt,  waffles)  ( vegetables)            0.014686   
771        ( yogurt,  waffles,  tortil

полученный датасет полностью опирается на связь с овощами так как если посмотреть овощи есть почти в каждой покупке

In [15]:
rules_apriori

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"( laundry detergent, cereals)",( vegetables),0.074099,0.596796,0.054740,0.738739,1.237842,0.010518,1.543299,0.207519
1,"( laundry detergent, cheeses)",( vegetables),0.070093,0.596796,0.050067,0.714286,1.196868,0.008235,1.411215,0.176884
2,"( yogurt, cheeses)",( vegetables),0.070761,0.596796,0.050734,0.716981,1.201384,0.008504,1.424655,0.180392
3,"( eggs, sandwich loaves)",( vegetables),0.060748,0.596796,0.043391,0.714286,1.196868,0.007137,1.411215,0.175124
4,"( laundry detergent, flour)",( vegetables),0.071429,0.596796,0.050067,0.700935,1.174497,0.007438,1.348214,0.160000
...,...,...,...,...,...,...,...,...,...,...
767,"( spaghetti sauce, yogurt, soap)",( vegetables),0.016021,0.596796,0.012016,0.750000,1.256711,0.002455,1.612817,0.207598
768,"( spaghetti sauce, yogurt, soda)",( vegetables),0.019359,0.596796,0.014019,0.724138,1.213377,0.002465,1.461615,0.179325
769,"( yogurt, tortillas, soda)",( vegetables),0.018692,0.596796,0.013351,0.714286,1.196868,0.002196,1.411215,0.167619
770,"( spaghetti sauce, yogurt, waffles)",( vegetables),0.014686,0.596796,0.010681,0.727273,1.218629,0.001916,1.478416,0.182080


In [16]:
rules_fpgrowth

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"( bagels, poultry, mixes)",( vegetables),0.029372,0.596796,0.021362,0.727273,1.218629,0.003832,1.478416,0.184835
1,"( ice cream, poultry, mixes)",( vegetables),0.022697,0.596796,0.017356,0.764706,1.281353,0.003811,1.713618,0.224674
2,"( poultry, mixes, lunch meat)",( vegetables),0.022029,0.596796,0.016021,0.727273,1.218629,0.002874,1.478416,0.183447
3,"( bagels, ice cream, individual meals)",( vegetables),0.020027,0.596796,0.014019,0.700000,1.172931,0.002067,1.344014,0.150448
4,"( ice cream, individual meals, soda)",( vegetables),0.018024,0.596796,0.014019,0.777778,1.303256,0.003262,1.814419,0.236962
...,...,...,...,...,...,...,...,...,...,...
767,"( individual meals, cereals, beef)",( vegetables),0.024032,0.596796,0.017356,0.722222,1.210167,0.003014,1.451535,0.177944
768,"( ice cream, aluminum foil, beef)",( vegetables),0.025367,0.596796,0.018024,0.710526,1.190569,0.002885,1.392887,0.164231
769,"( laundry detergent, aluminum foil, beef)",( vegetables),0.016021,0.596796,0.011348,0.708333,1.186894,0.001787,1.382415,0.160029
770,"( laundry detergent, yogurt, beef)",( vegetables),0.014686,0.596796,0.010681,0.727273,1.218629,0.001916,1.478416,0.182080


In [17]:

# Получение всех уникальных значений столбца 'consequents'
unic1 = rules_apriori['consequents'].unique()
unic2 = rules_fpgrowth['consequents'].unique()
# Вывод всех уникальных значений столбца 'consequents'
print(unic1)
print(unic2 )

[frozenset({' vegetables'})]
[frozenset({' vegetables'})]


In [18]:
data

,pork,sandwich bags,lunch meat,all- purpose,flour,soda,butter,vegetables,beef,aluminum foil,all- purpose.1,dinner rolls,shampoo,all- purpose.2
0,shampoo,hand soap,waffles,vegetables,cheeses,mixes,milk,sandwich bags,laundry detergent,dishwashing liquid/detergent,waffles,individual meals,hand soap,vegetables
1,pork,soap,ice cream,toilet paper,dinner rolls,hand soap,spaghetti sauce,milk,ketchup,sandwich loaves,poultry,toilet paper,ice cream,ketchup
2,juice,lunch meat,soda,toilet paper,all- purpose,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,pasta,tortillas,mixes,hand soap,toilet paper,vegetables,vegetables,paper towels,vegetables,flour,vegetables,pork,poultry,eggs
4,toilet paper,eggs,toilet paper,vegetables,bagels,dishwashing liquid/detergent,cereals,paper towels,laundry detergent,butter,cereals,bagels,paper towels,shampoo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1493,beef,sandwich bags,hand soap,paper towels,paper towels,all- purpose,beef,fruits,coffee/tea,beef,shampoo,mixes,cheeses,juice
1494,dinner rolls,lunch meat,spaghetti sauce,pasta,vegetables,cereals,dinner rolls,soap,milk,eggs,poultry,cheeses,cereals,mixes
1495,lunch meat,eggs,poultry,vegetables,tortillas,beef,beef,individual meals,dishwashing liquid/detergent,shampoo,dishwashing liquid/detergent,dishwashing liquid/detergent,hand soap,sugar
1496,ketchup,milk,poultry,cheeses,soap,toilet paper,yogurt,beef,waffles,sugar,spaghetti sauce,all- purpose,sandwich bags,toilet paper
